# Analysing Admissions Essays: Unsupervised Approaches using scikit-learn

There are two libraries that dominate text analysis in Python. The first is NLTK, which implements a range of natural language processing techniques (see other notebook).

The other dominant library is scikit-learn, which, at its most basic, provides a function to create a memory-efficient document-term matrix. It also implements a variety of quite sophisticated machine learning techniques that you can use on your text. It's a powerful library well suited for many purpouses.

Some of the approaches we will use below for our purpouses include:
* word weighting
* feature extraction
* text classification / supervised machine learning
    * L2 regression
    * classification algorithms such as nearest neighbors, SVM, and random forest
* clustering / unsupervised machine learning
    * k-means
    * pca
    * cosine similarity
    * LDA

Today, we'll start with the Document Term Matrix (DTM). The DTM is the bread and butter of most computational text analysis techniques, both simple and more sophisticated methods. In this lesson we will use Python's scikit-learn package learn to make a document term matrix from a .csv Music Reviews dataset (collected from MetaCritic.com). We will then use the DTM and a word weighting technique called tf-idf (term frequency inverse document frequency) to identify important and discriminating words within this dataset (utilizing the Pandas package). The illustrating question: what words distinguish reviews of Rap albums, Indie Rock albums, and Jazz albums? 

Finally, we will use the DTM to get an introduction to one method for uncovering patterns or themes within text: LDA, a topic modeling algorithm. Again, this will just be an introduction. Look for additional workshops in the future that will get into topic modeling in more detail.


### Outline
1. Import and view the data using Pandas
1. Explore the Data using Pandas
  1. Basic descriptive statistics
1. Creating the DTM: scikit-learn
  1. CountVectorizer function
1. What can we do with a DTM?
1. Tf-idf scores
  1. TfidfVectorizer function
1. Identifying Distinctive Words
  1. Application: Identify distinctive words by genre
1. Uncovering patterns using LDA

### Key Jargon
* *Document Term Matrix*:
  * a mathematical matrix that describes the frequency of terms that occur in a collection of documents. In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms.
* *TF-IDF Scores*: 
  *  short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.
* *Topic Modeling*:
  * A statistical model to uncover abstract topics within a text. It uses the co-occurrence fo words within documents, compared to their distribution across documents, to uncover these abstract themes. The output is a list of weighted words, which indicate the subject of each topic, and a weight distribution across topics for each document.
* *LDA*:
  * Latent Dirichlet Allocation. A implementation of topic modeling that assumes a Dirichlet prior. It does not take document order into account, unlike other topic modeling algorithms.
    
### Further Resources

[This blog post](https://de.dariah.eu/tatom/feature_selection.html) goes through finding distinctive words using Python in more detail 

Paper: [Fightin’ Words: Lexical Feature Selection and Evaluation for Identifying the Content of Political Conflict](http://languagelog.ldc.upenn.edu/myl/Monroe.pdf), Burt Monroe, Michael Colaresi, Kevin Quinn

[More detailed description of implementing LDA using scikit-learn](http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-topics-extraction-with-nmf-lda-py).

## 1. Import and view the data using Pandas

First, we read our corpus, which is stored as a .csv file on our hard drive, into a Pandas dataframe. 

Note: Pandas is great for data munging and basic calculations because it's so easy to use, and its data structure is really intuitive for me. It's not memory efficient however, so you might quickly need to move away from it. I recommend always always always using Pandas (or similar) over spreadsheets and Excel. [Excel is bad for science!](https://www.washingtonpost.com/news/wonk/wp/2016/08/26/an-alarming-number-of-scientific-papers-contain-excel-errors/)

### First we'll import, format, and view the data

Ths code brings in the packages we'll need to explore the data (Pandas and Nympy) before reading the data into a pandas dataframe and inspecting that data frame.

In [36]:
import pandas
import numpy
from io import StringIO


#this line opens the file, and the .read at the end reads the text contined into a variable as a string.
interview = open('/Volumes/Extra Space/Google Drive/Scholarship/Workshops - Talks - Notes/Comp Text Analysis - CTAWG/Computational Text Analysis Working Group/Smith Interview Project/Smith Interview Sample/Additions/SF657, 1 and 2 txt file.txt', 'r', encoding = 'utf_8').read()
#interview = interview.read()

#print(interview_string)

print("What type of variable is 'interview':", type(interview))
print(interview)

What type of variable is 'interview': <class 'str'>
Interview: SF657
I:	So, before we get started on the interview, I just kind of want to go over some logistics first. I guess the first question is can you hear me okay?
R:	I can hear you perfectly.
I:	Great. So the first thing is, would you be able to give me your address? We’re compensating our interviewees $40--
R:	So here’s what I’m going to ask you. I work [inaudible 1:26] program of recovery right now, and I see this as part of my service that I can make amends.
I:	Okay. 
R:	And I’d like you to donate that money, if you could really do this, is to The Safe House. The Safe House is actually a place in San Francisco that gets women off the street who have been in prostitution. If you could instead of giving it to me, if you guys could make a check out to them and give them money I would really appreciate it.
I:	That’s really wonderful of you. Yes, I believe we’ve had other respondents ask the same, and I believe that that is possib

Next we'll begin to parse the string so we can create a dataframe.

In [40]:
df = pandas.read_csv(interview, sep = '\s', encoding = 'utf_8')[1:6]

#df = pandas.read_csv(interview, )

#create a dataframe called "df"
#df = pandas.read_csv("/Volumes/Extra Space/Google Drive/Scholarship/Workshops - Talks - Notes/Comp Text Analysis - CTAWG/Computational Text Analysis Working Group/Smith Interview Project/Smith Interview Sample/Additions/SF657, 1 and 2 txt file.txt", sep = ',', encoding = 'utf_8')

# view the dataframe.  You can move the hashtag to view the other dataframe.
#df
# df2

/Users/benjamin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


OSError: [Errno 63] File name too long: 'Interview: SF657\nI:\tSo, before we get started on the interview, I just kind of want to go over some logistics first. I guess the first question is can you hear me okay?\nR:\tI can hear you perfectly.\nI:\tGreat. So the first thing is, would you be able to give me your address? We’re compensating our interviewees $40--\nR:\tSo here’s what I’m going to ask you. I work [inaudible 1:26] program of recovery right now, and I see this as part of my service that I can make amends.\nI:\tOkay. \nR:\tAnd I’d like you to donate that money, if you could really do this, is to The Safe House. The Safe House is actually a place in San Francisco that gets women off the street who have been in prostitution. If you could instead of giving it to me, if you guys could make a check out to them and give them money I would really appreciate it.\nI:\tThat’s really wonderful of you. Yes, I believe we’ve had other respondents ask the same, and I believe that that is pos

Next we can rename the colum headers so they are easier to work with.

In [2]:
# This renames the colum headers
df1.columns = ['CPID', 'College', 'PS1']
df2.columns = ['CPID', 'College', 'PS2']

# view the dataframe.  You can move the hashtag to view the other dataframe.
df1
# df2

,CPID,College,PS1
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th..."
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t..."
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...


Now we will merge the two dataframes on thier two common elements (CPID and College) using `merge`.

In [6]:
# Merge the two data frames so that we have one data frame with both questions attached to common CPID's and College.
df = pandas.merge(df1, df2, on=['CPID', 'College'])
df

,CPID,College,PS1,PS2
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ...","Since childhood, I have yearned for utopia. I ..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an..."


It can be helpful to see how much memory is being used by this new dataframe.  We can do that with the `info` option.

In [7]:
# Check the amount of memory being occupied by this newly created element.  
print(df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82574 entries, 0 to 82573
Data columns (total 4 columns):
CPID       82574 non-null int64
College    82574 non-null object
PS1        82544 non-null object
PS2        82542 non-null object
dtypes: int64(1), object(3)
memory usage: 436.7 MB
None


It is important to review data data that is contained in the new dataframe we created.  This code looks at the first essay in full.

In [4]:
#print the first essay from the column 'PS1'
df['PS1'][0]

'Bojio!\\\\That was what I playfully typed on my family\'s Whatsapp group chat after my older brother posted a picture of his and my sister in law\'s Bali resort. It was an expression that travelled from my mind to my flitting fingertips almost immediately. The resort was simply the image of serenity and solitude- and which student going through examination stress would not want to be a part of that?\\\\It was only when I got home and slumped on the sofa that I saw the nervous look on my mother\'s face. Her kohl-rimmed eyes were wide and her vermillion adorned forehead scrunched up as she asked, utterly confused, "What\'s bojio?"\\\\I burst out laughing. Sometimes, I forgot how every day brought around a new culture shock when you lived in a traditional Indian family but grew up in a multiracial community. The Hokkien phrase "bojio", literally meaning "never invite", is a popular colloquialism in Singapore to teasingly express annoyance at not being invited to something. My brother, wh

## 2. Explore the Data using Pandas

Let's first evaluate the general nature of the data to see if the ID's are unique, if there is any missing data, etc.  
We can also look at some descriptive statistics about this datasetto get a feel for what's in it. We'll do this using the Pandas package. 

Note: this is always good practice. It serves two purposes. It checks to make sure your data is correct, and there's no major errors. It also keeps you in touch with your data, which will help with interpretation. Love your data!

### Are the ID's Unique?

What ID's have more than one "PS1"s can be found by counting and ranking "ID"s

In [8]:
#This tells us if we have any duplicate IDs.  If each response is 1 we are ok.
print(df['CPID'].value_counts())

# This code seems to check for duplicate CPIDs.  If it's blank there are no duplicates.
print()
print("Array containing duplicate CPIDs:")
print(df.set_index('CPID').index.get_duplicates())

3016703    1
3002500    1
3146453    1
3152598    1
3019479    1
3042008    1
3039961    1
3018728    1
3035871    1
3123936    1
3125987    1
3115748    1
3030135    1
3117799    1
3009256    1
3013354    1
3130093    1
3136238    1
3003119    1
3027667    1
3029714    1
3068623    1
3056321    1
3084983    1
3107512    1
3105465    1
3109563    1
3103422    1
3101375    1
3058368    1
          ..
3044772    1
3014037    1
3040678    1
3042727    1
3151272    1
3153321    1
3012128    1
3132004    1
3028396    1
3007894    1
3132819    1
3048826    1
3132755    1
3134800    1
3061116    1
3059071    1
3136223    1
3100035    1
3110276    1
3021600    1
3085704    1
3130770    1
3081610    1
3144558    1
3093900    1
3095949    1
3089806    1
3091855    1
3132125    1
3116651    1
Name: CPID, dtype: int64

Array containing duplicate CPIDs:
[]


### Are there any missing Essays?

In [10]:
# This creates a variable empties

# First for PS1
print('Summarizing missing data for PS1:')
empties_PS1 = numpy.where(pandas.isnull(df['PS1']))[0]

print(empties_PS1)

# you notice that this is not formatted as a list.  The next opperation "list" gets it in the right format.
empties_PS1 = list(empties_PS1)
print(empties_PS1)

#This counts the number of missing essays for PS1.
print(len(empties_PS1))

#This lists the elemtns with missing data.
df.iloc[empties_PS1]

Summarizing missing data for PS1
[ 1776  3206  3566  6285  6801  7530  7930  8111  8571 11796 12977 15694
 19073 23667 24682 26014 28080 28573 29154 29548 31818 40212 41898 44980
 53738 64612 73519 74177 79276 81423]
[1776, 3206, 3566, 6285, 6801, 7530, 7930, 8111, 8571, 11796, 12977, 15694, 19073, 23667, 24682, 26014, 28080, 28573, 29154, 29548, 31818, 40212, 41898, 44980, 53738, 64612, 73519, 74177, 79276, 81423]
30


,CPID,College,PS1,PS2
1776,3133634,College of Engineering,NaN,NaN
3206,3157746,College of Letters and Science,NaN,NaN
3566,3041638,College of Letters and Science,NaN,NaN
6285,3108688,College of Natural Resources,NaN,I never described myself as super religious. W...
6801,3052354,College of Letters and Science,NaN,NaN
7530,3055366,College of Letters and Science,NaN,"At a young age the word ""weird "" or "" differen..."
7930,3056046,College of Engineering,NaN,NaN
8111,3001798,College of Letters and Science,NaN,NaN
8571,3145221,College of Natural Resources,NaN,"The ocean is my world. From an early age, the ..."
11796,3092946,College of Letters and Science,NaN,NaN


In [11]:
# Repeat the above steps for PS2
print('Summarizing missing data for PS2')
empties_PS2 = numpy.where(pandas.isnull(df['PS2']))[0]
print(empties_PS2)
empties_PS2 = list(empties_PS2)
print(empties_PS2)
print(len(empties_PS2))
df.iloc[empties_PS2]

Summarizing missing data for PS2
[ 1222  1776  3206  3566  3667  3900  6801  7930  8111 11796 12977 14552
 16926 19073 23667 24682 26014 28573 31818 36381 40212 41898 44980 53738
 57647 60910 62127 72379 73519 74177 79276 81423]
[1222, 1776, 3206, 3566, 3667, 3900, 6801, 7930, 8111, 11796, 12977, 14552, 16926, 19073, 23667, 24682, 26014, 28573, 31818, 36381, 40212, 41898, 44980, 53738, 57647, 60910, 62127, 72379, 73519, 74177, 79276, 81423]
32


,CPID,College,PS1,PS2
1222,3152303,College of Letters and Science,"Every day, the mirror reminds me that I carry ...",NaN
1776,3133634,College of Engineering,NaN,NaN
3206,3157746,College of Letters and Science,NaN,NaN
3566,3041638,College of Letters and Science,NaN,NaN
3667,3162451,College of Natural Resources,DAD'S BACK ACHED BECAUSE OF ME. Mom's hands mo...,NaN
3900,3060413,College of Letters and Science,"I am lying down on my belly, savoring the exqu...",NaN
6801,3052354,College of Letters and Science,NaN,NaN
7930,3056046,College of Engineering,NaN,NaN
8111,3001798,College of Letters and Science,NaN,NaN
11796,3092946,College of Letters and Science,NaN,NaN


Drop the missing data using.

In [29]:
empties_any = empties_PS1 + list(set(empties_PS2) - set(empties_PS1))
empties_any.sort()

print(empties_any)
print(len(empties_any))

[1222, 1776, 3206, 3566, 3667, 3900, 6285, 6801, 7530, 7930, 8111, 8571, 11796, 12977, 14552, 15694, 16926, 19073, 23667, 24682, 26014, 28080, 28573, 29154, 29548, 31818, 36381, 40212, 41898, 44980, 53738, 57647, 60910, 62127, 64612, 72379, 73519, 74177, 79276, 81423]
40


In [30]:
df_no_missing = df.drop(df.index[empties_any])

# df_no_missing = df.dropna()
df_no_missing

,CPID,College,PS1,PS2
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ...","Since childhood, I have yearned for utopia. I ..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an..."


## 3. Creating a smaller sample to check the code

In this section we'll create a smaller sample of the code to that the analysis we construct below works.

In [88]:
# this generates a random sample of N essays, with a random state of 0 for reproducability.
# might want to slowly increase this number to see when it slows down
df_sample = df_no_missing.sample(n=5000, random_state=0)

df_sample = df_sample.sort_index()

df_sample = df_sample.reset_index()

df_sample

,index,CPID,College,PS1,PS2
0,48,3140324,College of Letters and Science,I let out a deep sigh of disappointment when I...,I stepped out of a decorated rickshaw only to ...
1,51,3011169,College of Letters and Science,"I am from a family of , I am eldest son of the...","I want to be a better person, a person with gr..."
2,54,3059554,College of Letters and Science,"On the tiny desk in front of me, a yellow, pen...","I power off my phone, press my hand to the doo..."
3,56,3031745,College of Letters and Science,The two people who have shaped me most are my ...,"Stumbling through the park at lightning speed,..."
4,60,3060991,College of Letters and Science,I easily have one of the most competitive fami...,Having gone through the troubles that I went t...
5,128,3071955,College of Letters and Science,"It was the week of June , when I found out my...",Giving back to my community has always been im...
6,151,3104723,College of Natural Resources,Despite prevailing opinions to the exact oppos...,For 13 years I think I hardly spoke a handful ...
7,166,3094246,College of Letters and Science,"""Good morning"", I said. Eight pairs of eyes st...","Surrounded by mountains of canned goods, used ..."
8,181,3130640,College of Letters and Science,It is strange how the social environment influ...,"""Last Ketchup"" is the first song I truly loved..."
9,213,3020042,College of Engineering,"""Computers think and perform tasks like humans...","""The SAT is a test that you cannot study for b..."


This code is not critical now.  It was used before to check to see if there were any empties contined in the sample to see if that broke the code below.  Now that empies have been removed, this is redundant so I've commented it out.

In [32]:
# empties_sample = numpy.where(pandas.isnull(df_sample['PS1']))

# #The len command seems not to be counting what I'm after.
# print(len(empties_sample))
# print(empties_sample)

## 4. Creating the DTM: scikit-learn

Ok, that's the summary of the metadata. Next, we turn to analyzing the text of the reviews. Remember, the text is stored in the 'body' column. First, a preprocessing step to remove numbers.

In [33]:
# This gets rid of numbers - THIS SEEMS TO BREAK WITH MY DATA
# It appears that if you drop rows with missing data it works!
df_no_missing['PS1'] = df_no_missing['PS1'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))

# If this generates an error we might consider uncommeting code below to see the data type:
#df.dtypes

Our next step is to turn the text into a document term matrix using the scikit-learn function called CountVectorizer. There are two ways to do this. We can turn it into a sparse matrix type, which can be used within scikit-learn for further analyses.  We can then turn it into a full documnet term matrix, but this is very memory intensive and might not be a great idea for larger data sets.

In [40]:
#import the function CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

countvec = CountVectorizer()

# Original sklearn_dtm = CountVectorizer().fit_transform(df.PS1)
#I added the values.astype section below and it seemed to fix the count vectorizer issues
sklearn_dtm_PS1 = countvec.fit_transform(df_no_missing['PS1'].values.astype('U'))
sklearn_dtm_PS2 = countvec.fit_transform(df_no_missing['PS2'].values.astype('U'))

print('PS1 sparse matrix type')
print(sklearn_dtm_PS1)
print(' ')
print('PS2 sparse matrix type')
print(sklearn_dtm_PS2)

PS1 sparse matrix type
  (0, 11093)	4
  (0, 97120)	8
  (0, 105895)	9
  (0, 106651)	3
  (0, 74210)	1
  (0, 100721)	1
  (0, 68478)	6
  (0, 64424)	26
  (0, 33417)	3
  (0, 106660)	1
  (0, 40307)	1
  (0, 16189)	1
  (0, 1747)	1
  (0, 68281)	1
  (0, 12517)	2
  (0, 75275)	1
  (0, 73471)	1
  (0, 68009)	12
  (0, 43506)	1
  (0, 3727)	19
  (0, 88884)	1
  (0, 46552)	10
  (0, 54702)	1
  (0, 7571)	1
  (0, 81452)	2
  :	:
  (82533, 86535)	1
  (82533, 89245)	1
  (82533, 76850)	1
  (82533, 63558)	1
  (82533, 963)	1
  (82533, 24568)	2
  (82533, 58567)	1
  (82533, 76933)	1
  (82533, 88299)	1
  (82533, 16211)	1
  (82533, 7654)	1
  (82533, 42345)	2
  (82533, 95953)	1
  (82533, 56816)	1
  (82533, 18483)	1
  (82533, 63466)	1
  (82533, 17931)	1
  (82533, 82414)	1
  (82533, 30347)	1
  (82533, 36670)	1
  (82533, 22701)	1
  (82533, 95698)	1
  (82533, 22328)	1
  (82533, 46329)	1
  (82533, 67842)	1
 
PS2 sparse matrix type
  (0, 23768)	1
  (0, 101240)	1
  (0, 6489)	9
  (0, 98539)	1
  (0, 90041)	1
  (0, 73096)	1
  (0

This format is called Compressed Sparse Format. It save a lot of memory to store the dtm in this format, but it is difficult to look at for a human. To illustrate the techniques in this lesson we will first convert this matrix back to a Pandas dataframe, a format we're more familiar with. For larger datasets, you will have to use the Compressed Sparse Format. Putting it into a DataFrame, however, will enable us to get more comfortable with Pandas! For this data, we will skip this step to avoid crashing the kernal.

In [37]:
# #we do the same as we did above, but covert it into a Pandas dataframe. Note this takes quite a bit more memory, so will not be good for bigger data.
# dtm_df = pandas.DataFrame(countvec.fit_transform(df_sample['PS1'].values.astype('U')).toarray(), columns=countvec.get_feature_names(), index = df_sample.index)

# #view the dtm dataframe
# dtm_df

## 5. What can we do with a DTM?

We can do a number of calculations using a DTM. For a toy example, we can quickly identify the most frequent words (compare this to how many steps it took in lesson 2, where we found the most frequent words using NLTK).

In [38]:
print(dtm_df.sum().sort_values(ascending=False))
# print(dtm_df.sum().sort_values(ascending=False))

AttributeError: 'numpy.int64' object has no attribute 'sort_values'

In [ ]:
#####Exercise:
###Print out the most infrequent words rather than the most frequent words.
##Gold star challenge: print the average number of times each word is used in an essay
print(dtm_df.mean().sort_values(ascending=False))
#Print this out sorted from highest to lowest.

What else does the DTM enable? Because it is in the format of a matrix, we can perform any matrix algebra or vector manipulation on it, which enables some pretty exciting things (think vector space and Euclidean  geometry). But, what do we lose when we reprsent text in this format?

Today, we will use variations on the DTM to find distinctive words in this dataset, and then do some preliminary work discovering themes in text.

## 6. Tf-idf scores

How to find distinctive words in a corpus is a long-standing question in text analysis. We saw a few ways to this yesterday, using natural language processing. Today, we'll learn one simple approach to this: word scores. The idea behind words scores is to weight words not just by their frequency, but by their frequency in one document compared to their distribution across all documents. Words that are frequent, but are also used in every single document, will not be distinguising. We want to identify words that are unevenly distributed across the corpus.

One of the most popular ways to weight words (beyond frequency counts) is *tf-idf* scores. By offsetting the frequency of a word by its document frequency (the number of documents in which it appears) will in theory filter out common terms such as 'the', 'of', and 'and'.

More precisely, the inverse document frequency is calculated as such:

number_of_documents / number_documents_with_term

so:

tfidf_word1 = word1_frequency_document1 * (number_of_documents / number_document_with_word1)

You can, and often should, normalize the numerator: 

tfidf_word1 = (word1_frequency_document1 / word_count_document1) * (number_of_documents / number_document_with_word1)

We can calculate this manually, but scikit-learn has a built-in function to do so. We'll use it, but a challenge for you: use Pandas to calculate this manually. 

To do so, we simply do the same thing we did above with CountVectorizer, but instead we use the function TfidfVectorizer.

In [ ]:
#import the function
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvec = TfidfVectorizer()

# #create the dtm, but with cells weigthed by the tf-idf score.
# dtm_tfidf_df = pandas.DataFrame(tfidfvec.fit_transform(df.PS1).toarray(), columns=tfidfvec.get_feature_names(), index = df.index)

# #view results
# dtm_tfidf_df

Let's look at the 20 words with highest tf-idf weights.

In [ ]:
# print(dtm_tfidf_df.max().sort_values(ascending=False)[0:20])

## 7. Uncovering Patterns: LDA

Frequency counts and tf-idf scores are done at the word level. There are other methods of exporatory or unsupervised analysis on the document level and by examining the co-occurrence of words within documents. Scikit-learn allows for many of these methods, including:

* document clustering
* document or word similarities using cosine similarity
* pca
* topic modeling

We'll run through an example of topic modeling here. Again, the goal is not to learn everything you need to know about topic modeling. Instead, this will provide you some starter code to run a simple model, with the idea that you can use this base of knowledge to explore this further.

We will run Latent Dirichlet Allocation, the most basic and the oldest version of topic modeling. We will run this in one big chunk of code. Our challenge: use our knowledge of scikit-learn that we gained aboe to walk through the code to understand what it is doing. Your challenge: figure out how to modify this code to work on your own data, and/or tweak the parameters to get better output.

Note: we will be using a different dataset for this technique. The music reviews in the above dataset are often short, one word or one sentence reviews. Topic modeling is not really appropriate for texts that are this short. Instead, we want texts that are longer and are composed of multiple topics each. For this exercise we will use a database of children's literature from the 19th century. 

The data were compiled by students in this course: http://english197s2015.pbworks.com/w/page/93127947/FrontPage
Found here: http://dhresourcesforprojectbuilding.pbworks.com/w/page/69244469/Data%20Collections%20and%20Datasets#demo-corpora

That page has additional corpora, for those interested in exploring text analysis further.

I did some minimal cleaning to get the children's literature data in .csv format for our use.

In [93]:
# df_lit = pandas.read_csv("/Volumes/Extra Space/Google Drive/Scholarship/Writing Projects - Personal/Admissions Essays/Small Sample/AdmissionsEssays/statement_test_031417.csv", sep = ',', encoding = 'utf-8')

# #drop rows where the text is missing. I think there's only one row where it's missing, but check me on that.
# df_lit = df_lit.dropna(subset=['PS1'])

df_lit = df_no_missing

#view the dataframe
df_lit

,CPID,College,PS1,PS2
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...
1,3092833,College of Engineering,"My world is shaped by a ' "", lean man, who loo...","Since childhood, I have yearned for utopia. I ..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an..."


Now we're ready to fit the model. This requires the use of CountVecorizer, which we've already used, and the scikit-learn function LatentDirichletAllocation.

See [here](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) for more information about this function. 

In [94]:
import time
start_time = time.time()

#should switch to batch (from online).  n_samples should be closer to full set

####Adopted From: 
#Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

n_samples = 2000
n_topics = 5
n_top_words = 50

##This is a function to print out the top words for each topic in a pretty way.
#Don't worry too much about understanding every line of this code.
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

# Use tf-idf features
tfidf_vectorizer = TfidfVectorizer(max_df=0.80, min_df=50,
                                   max_features=None,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(df_lit['PS1'])

# Use tf (raw term count) features
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.80, min_df=50,
                                max_features=None,
                                stop_words='english'
                                )

tf = tf_vectorizer.fit_transform(df_lit['PS1'])

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_topics=%d..."
      % (n_samples, n_topics))

#define the lda function, with desired options TAKE A LOOK AT THIS.  MIGHT BE TOO FEW
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=20,  #100 (THATS WHAT LAURA DID)
                                learning_method='online',  # CHANGE THIS TO BATCH
                                learning_offset=80.,
                                total_samples=n_samples, # TAKE A LOOK AT THIS RE CHANGE TO BATCH
                                random_state=0)
#fit the model
lda.fit(tf)

#print the top words per topic, using the function defined above.
#Unlike R, which has a built-in function to print top words, we have to write our own for scikit-learn
#I think this demonstrates the different aims of the two packages: R is for social scientists, Python for computer scientists

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

print("This program took", time.time() - start_time, "seconds to run.")

Extracting tf features for LDA...
Fitting LDA models with tf features, n_samples=2000 and n_topics=5...

Topics in LDA model:

Topic #0:
science world computer engineering math knowledge school technology research work new passion like business time learning learn field problems physics create father art study future learned use engineer make high curiosity design class chemistry love problem biology project experience different programming mathematics career books pursue way interests things skills working

Topic #1:
family life parents school mother want work father help people hard time make education college mom like did able years know dad world better person future things way high best day come brother home just wanted dreams going dream taught sister learned support live good career ve working didn year

Topic #2:
school high students year team community class time music friends new student people club years life learned grade work group classes best experience program like acti

In [ ]:
####Exercise:
###Run the same code as above but change some of the parameters. How does this change the output.
###Suggestions:
## 0. Use tf-idf scores rather than raw counts. (hint: look for the variable name we created) 
## 1. Change the number of topics. What do you find?
## 2. Do not remove stop words. How does this change the output?

One thing we may want to do with the output is find the most representative texts for each topic. A simple way to do this (but not memory efficient), is to merge the topic distribution back into the Pandas dataframe.

First get the topic distribution array.

In [95]:
topic_dist = lda.transform(tf)
topic_dist

array([[   0.20343325,    0.20504045,    0.20497347,  110.28843997,
         139.09811286],
       [ 109.96753759,   30.80927003,   11.87385879,   21.3098719 ,
          83.03946168],
       [  41.44346225,    0.20524765,   70.8687638 ,   46.7489046 ,
          62.7336217 ],
       ..., 
       [   4.32063697,    0.20647573,   98.60036012,    0.20334955,
          69.66917763],
       [   0.20494707,   18.87963714,   50.58311541,   74.04510118,
          42.2871992 ],
       [   0.20368881,    0.20459388,    8.71260145,   70.13343541,
         116.74568044]])

Merge back in with the original dataframe.

In [96]:
topic_dist_df = pandas.DataFrame(topic_dist)
df_w_topics = topic_dist_df.join(df_lit)
df_w_topics

,0,1,2,3,4,CPID,College,PS1,PS2
0,0.203433,0.205040,0.204973,110.288440,139.098113,3128896.0,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...
1,109.967538,30.809270,11.873859,21.309872,83.039462,3092833.0,College of Engineering,"My world is shaped by a ' "", lean man, who loo...","Since childhood, I have yearned for utopia. I ..."
2,41.443462,0.205248,70.868764,46.748905,62.733622,3142974.0,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...
3,0.202537,52.873869,31.534691,59.120310,46.268593,3020517.0,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...
4,10.124690,53.875667,33.511001,27.281887,0.206755,3121294.0,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...
5,0.206644,6.189918,25.058109,141.975725,65.569604,3162551.0,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins..."
6,11.736947,138.640419,0.204997,0.209768,65.207869,3090215.0,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...
7,117.207170,0.206436,57.122360,35.257507,0.206528,3158702.0,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...
8,6.245896,90.026427,57.320493,0.203186,0.203998,3095465.0,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,..."
9,60.436144,58.569313,0.204069,0.206332,49.584141,3147750.0,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an..."


In [97]:
df_w_topics.columns = ['T0_PS1', 'T1_PS1', 'T2_PS1', 'T2_PS1', 'T4_PS1', 'CPID', 'College', 'PS1', 'PS2']
df_w_topics

,T0_PS1,T1_PS1,T2_PS1,T2_PS1,T4_PS1,CPID,College,PS1,PS2
0,0.203433,0.205040,0.204973,110.288440,139.098113,3128896.0,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...
1,109.967538,30.809270,11.873859,21.309872,83.039462,3092833.0,College of Engineering,"My world is shaped by a ' "", lean man, who loo...","Since childhood, I have yearned for utopia. I ..."
2,41.443462,0.205248,70.868764,46.748905,62.733622,3142974.0,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...
3,0.202537,52.873869,31.534691,59.120310,46.268593,3020517.0,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...
4,10.124690,53.875667,33.511001,27.281887,0.206755,3121294.0,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...
5,0.206644,6.189918,25.058109,141.975725,65.569604,3162551.0,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins..."
6,11.736947,138.640419,0.204997,0.209768,65.207869,3090215.0,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...
7,117.207170,0.206436,57.122360,35.257507,0.206528,3158702.0,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...
8,6.245896,90.026427,57.320493,0.203186,0.203998,3095465.0,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,..."
9,60.436144,58.569313,0.204069,0.206332,49.584141,3147750.0,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an..."


In [98]:
df_w_topics.to_csv('Admissions_PS1_Full_Output.csv', sep=',')

Now we can sort the dataframe for the topic of interest, and view the top documents for the topics.
Below we sort the documents first by Topic 0 (looking at the top words for this topic I think it's about family, health, and domestic activities), and next by Topic 1 (again looking at the top words I think this topic is about children playing outside in nature). These topics may be a family/nature split?

Look at the titles for the two different topics. Look at the gender of the author. Hypotheses?

In [ ]:
print(df_w_topics[['ID', 'PS1', 0]].sort_values(by=[0], ascending=False))

We can read individual essays in full using the code below.  Change the number in the final set of brackets to point to a spesific serial number (ID-1).

In [ ]:
df['PS1'][1980]

In [ ]:
print(df_w_topics[['ID', 'PS1', 1]].sort_values(by=[1], ascending=False))

In [ ]:
df['PS1'][2131]

In [ ]:
print(df_w_topics[['ID', 'PS1', 2]].sort_values(by=[2], ascending=False))

In [ ]:
df['PS1'][3515]

In [ ]:
print(df_w_topics[['ID', 'PS1', 3]].sort_values(by=[3], ascending=False))

In [ ]:
df['PS1'][2645]

In [ ]:
print(df_w_topics[['ID', 'PS1', 4]].sort_values(by=[4], ascending=False))

In [ ]:
df['PS1'][811]

What other patterns might we find with topic modeling? Toward what end?

In [ ]:
###Ex (gold star exercise!): 
#       Find the most prevalent topic in the corpus.
#       Find the least prevalent topic in the corpus. 
#       Find the most prevalent topic by the gender of the author.
#       Hint: How do we define prevalence? What are different ways of measuring this,
#              and the benefits/drawbacks of each.


#       Extra bonus gold star exercise:
#          This topic model provide the topic distribtution for 127 rows, but there are 131 rows in the full data.
#          What is going on here? (I don't have an answer to this. I hope someone can figure it out!)           